# 8기 과제 - 딥러닝 기반 상품 카테고리 자동 분류 서버

## 과제 개요
* 출제자 : 남상협 멘토 (justin@buzzni.com) / 버즈니 (http://buzzni.com) 대표
* 배경 : 카테고리 분류 엔진은 실제로 많은 서비스에서 사용되는 중요한 기계학습 기술이다. 본 과제의 주제는 버즈니 개발 인턴이자 마에스트로 6기 멘티가 아래와 나와 있는 기본 분류 모델을 기반으로 deep learning 기반의 feature 를 더해서 고도화된 분류 엔진을 만들어서 2016 한국 정보과학회 논문으로도 제출 했던 주제이다. 기계학습에 대한 학습과, 실용성 두가지 측면에서 모두 도움이 될 것으로 보인다.


## 과제 목표
* 입력 : 상품명, 상품 이미지
* 출력 : 카테고리
* 목표 : 가장 높은 정확도로 분류를 하는 분류 엔진을 개발



## 평가 항목 
* 성능평가 (100%)
 
## 제출 항목 
* 채점 서버에 자신이 분류한 class id 리스트를 파라미터로 넣어서 호출한다. 
* name - 자신의 이름을 넣는다. 실제 점수판에는 공개가 안됨, 추후 평가시에 일치하는 이름의 멘티 점수로 사용함. 요청한 평가 중에서 가장 높은 점수의 평가 점수로 업데이트됨.
* nickname - 점수판에 공개되는 이름, 자신의 이름으로 해도 되고, 닉네임으로 해도 됨. 구분을 위해서 사용하는 feature(text, textimage) 와 알고리즘 (svm, cnn) 등을 닉네임 뒤에 붙여준다. 
* pred_list - 분류한 카테고리 id 리스트를 , 로 묶은 데이터 
* 평가 점수가 반환된다. - precision, 높을 수록 좋다. 두가지 방법 각각 50%씩 점수 반영 
* mode - 'test' 로 호출하면 웹으로 순위가 공개되는 테스트 평가를 수행하고 결과 점수가 반환된다. 해당 결과 점수는 http://eval.buzzni.net:20002/score 에서 확인 가능함. 실제 성적 평가는 'eval' 로 평가용 데이터로 호출하면 된다. 이때는 점수가 반환되거나, 웹 점수 보드에도 나오지 않는다. 
* 너무 자주 평가를 요청하기 보다, 가급적 자체적으로 평가 해서, 괜찮게 나올때 요청하길 권장 
```python
import requests
name='test1'
nickname='test1_text_svm'
mode='test' #'eval' 을 실제 성적 평가용. 분류 점수 반환 안됨.
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,"mode":mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())         
         ```

## 성능 향상 포인트
* http://localhost:8000/notebooks/maestro8_deeplearning_product_classifier.ipynb 이 노트북에 있는 딥러닝 기반의 분류기로 분류할 경우에 더 높은 성능을 낼 수 있어서 유리함
* 아래의 방법들은 하나의 예이고, 아래에 나와 있지 않은 다양한 방법들도 가능함.
* 전처리 
 * 오픈된 형태소 분석기(예 - konlpy) 를 써서, 단어 띄어쓰기를 의미 단위로 띄어서 학습하기
 * bigram, unigram, trigram 등 단어 feature 를 더 다양하게 추가하기
* 딥러닝 
 * embedding weight 를 random 이 아닌 학습된 값을 사용하기 (https://radimrehurek.com/gensim/models/word2vec.html)
 * 이미지 feature 를 CNN으로 추출할때 더 성능이 좋은 모델 사용하기 (예제로 준 데이터는 mobilenet 으로 성능보다 속도 위주로 된 모델)
 * 다양한 파라미터(hyper parameter) 로 실험 해보기 
* 피쳐 조합  
 * 이미지 feature 와 text feature 를 합치는 부분 잘하기 


## 평가 점수 서버 
* 현재 평가 순위를 json 형태로 반환한다.
* 여러번 호출했을때는 가장 높은 점수로 업데이트 한다.
 * http://eval.buzzni.net:20002/score
* 실제 점수는 

In [2]:
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
from keras import backend
from keras.layers import Dense, Input, Lambda, LSTM, TimeDistributed
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model



/usr/local/lib/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


### 파일에서 학습 데이터를 읽는다.

In [3]:
import json


In [4]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [5]:
# joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [6]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [7]:
print(y_name_id_dict)

{'취미': 5, '스포츠/레저': 9, '가구/인테리어': 11, '가전': 12, '의류': 2, '여행/e쿠폰': 15, '생필품/주방': 13, '건강': 7, '식품': 6, '반려동물': 4, '디지털': 10, '잡화': 14, '뷰티': 0, '출산/육아': 16, '컴퓨터': 3, '도서/문구': 8, '자동차/공구': 1}


In [8]:

# y_name_set = set(y_text_list)
# y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
# y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_id_list = [y_name_id_dict[x] for x in y_text_list]
print(y_id_list)

[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,

### text 형태로 되어 있는 상품명을 각 단어 id 형태로 변환한다.

In [9]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(map(lambda i : i[1],x_text_list))
y_list = [y_name_id_dict[x] for x in y_text_list]
print(vectorizer.transform([u'노트북 가방']))
print("----")
print(vectorizer.transform([u'노트북 가방 받침대']))

  (0, 9305)	1
  (0, 12098)	1
----
  (0, 9305)	1
  (0, 12098)	1
  (0, 16632)	1


### train test 분리하는 방법 

In [10]:
print(x_text_list[2])

('1546650266', '(AGCRIP 웨빙 포켓조끼 빅사이즈 조끼 MK321_324 아웃')


In [11]:
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)


In [12]:
# print(vectorizer.transform(list(map(lambda i : i[1],X_train))))

### 몇개의 파라미터로 간단히 테스트 하는 방법

In [13]:

for c in [1,5,10]:
    clf = LinearSVC(C=c)
    X_train_text = map(lambda i : i[1],X_train)
    clf.fit(vectorizer.transform(X_train_text), y_train)
    print (c,clf.score(vectorizer.transform(map(lambda i : i[1],X_test)), y_test))


1 0.6
5 0.599411764706
10 0.599411764706


### 최적의 파라미터를 알아서 다 해보고, n-fold cross validation까지 해주는 방법 - GridSearchCV

In [14]:
svc_param = np.logspace(-1,1,4)

In [15]:

gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)

In [16]:
gsvc.fit(vectorizer.transform(map(lambda i : i[1],x_text_list)), y_list)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([  0.1    ,   0.46416,   2.15443,  10.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [17]:
print(gsvc.best_score_, gsvc.best_params_)

0.565764705882353 {'C': 0.10000000000000001}


#### 평가 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송

In [18]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [19]:
pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [20]:
print (pred_list.tolist())

[1, 1, 2, 16, 1, 1, 5, 1, 13, 13, 8, 0, 6, 11, 2, 1, 0, 3, 8, 9, 8, 12, 13, 10, 15, 9, 10, 4, 3, 3, 2, 13, 6, 1, 3, 4, 16, 2, 4, 5, 14, 15, 5, 16, 4, 3, 0, 11, 13, 4, 4, 2, 12, 15, 14, 0, 3, 14, 2, 3, 10, 6, 1, 2, 10, 0, 12, 10, 2, 0, 3, 4, 2, 5, 7, 10, 5, 2, 3, 11, 9, 13, 3, 7, 3, 6, 5, 1, 14, 16, 15, 4, 2, 6, 6, 6, 12, 7, 4, 6, 2, 8, 7, 0, 5, 2, 7, 0, 13, 6, 13, 14, 9, 13, 8, 7, 2, 2, 12, 10, 3, 6, 11, 5, 6, 11, 16, 2, 16, 14, 16, 1, 4, 10, 11, 0, 15, 14, 11, 5, 6, 13, 15, 16, 7, 0, 15, 10, 10, 2, 9, 0, 9, 10, 9, 9, 7, 3, 7, 6, 15, 2, 10, 3, 8, 2, 15, 16, 11, 13, 11, 2, 3, 0, 13, 12, 8, 15, 14, 5, 14, 13, 11, 15, 15, 3, 5, 3, 11, 15, 7, 5, 10, 3, 2, 15, 14, 0, 13, 13, 16, 2, 6, 12, 8, 7, 12, 15, 4, 4, 11, 13, 9, 12, 6, 5, 4, 14, 6, 7, 5, 8, 16, 3, 13, 9, 1, 6, 14, 1, 7, 4, 0, 10, 8, 6, 3, 4, 16, 7, 16, 7, 0, 4, 2, 7, 8, 15, 15, 15, 2, 14, 6, 4, 10, 11, 6, 9, 1, 4, 13, 15, 9, 0, 7, 7, 6, 0, 14, 14, 11, 11, 16, 16, 1, 8, 0, 16, 16, 9, 15, 3, 4, 13, 13, 8, 3, 8, 5, 11, 12, 7, 6, 14, 13,

In [21]:
import requests
name='성주녕'
nickname='성주녕_text_svm'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


{'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.', 'precision': 0.6393188854489165}


#### eval 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송
 * 실제 여기서 나온 점수로 채점을 한다.

In [22]:
eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))
pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
name='test0'
nickname='test0_text_svm'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


{'msg': 'success', 'precision': ''}


### CNN 으로 추출한 이미지 데이터 사용하기 
 * keras mobilenet 으로 추출한 데이터, 이 데이터를 아래처럼 읽어서 사용 가능함
 * 더 성능이 높은 모델로 이미지 피쳐를 추출하면 성능 향상 가능함 

In [24]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
        if idx%100 == 0:
            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
        print (line)
        break
        

0
1234621373 2.6489890387892956e-06,3.405012466828339e-05,4.848148364544613e-06,6.605810995097272e-06,1.476716988690896e-05,2.5480754629825242e-05,8.004099981917534e-06,8.715509807188937e-07,9.519604873275966e-07,1.9165463527315296e-06,3.8953160697019484e-07,6.25218888217205e-07,1.2698757245743764e-06,1.7236279745702632e-06,2.8134704734839033e-06,1.2079543694198946e-06,2.1540823524901498e-07,2.312025571882259e-06,3.553984981863323e-07,4.418127730332344e-07,7.728485940106111e-08,1.0770256722025806e-06,8.766759833633841e-07,8.917487548387726e-07,7.151564886953565e-07,6.351970569085097e-06,5.283235077513382e-05,0.0003902010794263333,1.5194400475593284e-05,0.00031568240956403315,1.18185744213406e-06,2.173033863073215e-05,7.622767839166045e-07,4.079382051713765e-06,2.7062740173278144e-06,1.1413741049182136e-05,4.8681566113373265e-05,2.6313375656172866e-07,1.847035309765488e-05,2.0308896182541503e-06,7.38462622393854e-05,0.00011853341129608452,9.790997864911333e-06,1.3620918934975634e-06,0.0

In [85]:
from scipy import sparse 

In [86]:
img_feature_list = []
for pid, name in X_train:
#     print(pid, name)
    if pid in pid_img_feature_dict:
        img_feature_list.append(pid_img_feature_dict[pid])
#         print (len(pid_img_feature_dict[pid]),type(pid_img_feature_dict[pid]))
#         break
    else:
        img_feature_list.append(np.zeros(1000))
#     break

In [87]:
img_feature_test_list = []
for pid, name in X_test:
    if pid in pid_img_feature_dict:
        img_feature_test_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_test_list.append(np.zeros(1000))


In [88]:
print(len(img_feature_list))

6800


#### 아래 부분은 text feature 와 이미지 feature 를 합쳐서 feature 를 만드는 부분이다. 이 부분에 대해서는 각자 한번 합치는 방법을 찾아 보면 된다. 

In [25]:
print(X_train)
# concat_x_list = func((vectorizer.transform(map(lambda i : i[1],X_train)), img_feature_list))
# concat_test_x_list = func((vectorizer.transform(map(lambda i : i[1],X_test)), img_feature_test_list))


[('1284373589', '도그차일드 레오파드애견신발 노랑 강아지슈즈 애'), ('1603361243', '불스 방청윤활제 360ML 윤활방청제 방청유 윤활유 녹'), ('1086777272', '[쿠스쿠스파이]블루베리치즈파이 800g(100g x 8조각)'), ('1367029764', '파사바체 유리 계량컵/주방용품/계량컵/계량용컵/조리도구/계량용품'), ('1687461732', '제주 코코몽에코파크+메이즈랜드  제주도 관광지 2곳 패키지 입장권 할인 /기프트제주/제주 승마체험/제주 승마/제주승마할인/제주 승마장 추천/제주도 승마체험 가격말타는곳 말타기'), ('1639169754', 'BA537 발목 서포트 LP-954 압박밴드 - 발목보호대 무'), ('1470415954', '[중고]서버용 HDD SAS Seagate ST300MP0005 300GB/15K'), ('1255743159', '(투핫) [헬로키티]헬로키티 쿠킹컵 세트'), ('1611544124', '인테리어 스위치커버 그래픽스티커 파인애플 WBSS7170'), ('1486501072', '멋스러운 캐주얼 정장 서스펜더 멜빵 TE1005'), ('1549175351', '1200M [미코아이엔티] 아트조이 DIY 명화그리기 숲속의 작은집'), ('1401228966', '3M  PF 프라이버시 필터 (15.6W)'), ('1530257201', 'LD-PA5 인도 사우디아라비아 등 해외여행 아답타'), ('1321872342', '[착한스포츠]STAR 고무연식 야구공(12개입)'), ('1578581418', '다람쥐PMC-230 고양이장난감 고양이용장난감 캣장난감'), ('1036567080', 'PS무료배송 [랄라룹시 윈터드레스패션세트(506539)] (인형별매) 랄라룹시 윈터드레스 패션세트 인형옷 인..'), ('1651852477', '보브 듀얼 커버 CC크림 30ml 21호 라이트 톤업 커버'), ('1629587718', '허니버터링쿠키(요쿠르트)국내산쿠키 X2개 강아지과

In [28]:


for c in [1]:
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_x_list, y_train)
    print (c,clf2.score(concat_test_x_list, y_test))


NameError: name 'concat_x_list' is not defined

In [ ]:
del pid_img_feature_dict

### CNN 피쳐를 추가 해서 분류후 평가 서버에 분류 결과를 전송 

In [ ]:
pid_img_feature_dict = {}
with open("/workspace/resources/refined_category_dataset.img_feature.eval.dat") as fin:
    for idx,line in enumerate(fin):
        if idx%100 == 0:
            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#         print (line)
#         break
        

In [ ]:
test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        test_x_text_list.append((info['pid'],info['name']))

# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [ ]:
img_feature_eval_list = []

In [ ]:
for pid, name in test_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))


In [ ]:
print (len(img_feature_eval_list), len(eval_x_text_list))

In [ ]:
x_feature_list = vectorizer.transform(map(lambda i : i[1],test_x_text_list))

#### 2개 feature 를 합치는 방법 찾아보기 

In [ ]:
concat_test_x_list = sparse.hstack((x_feature_list, img_feature_eval_list),format='csr')

In [ ]:
pred_list = clf2.predict(concat_test_x_list)

In [ ]:

import requests
name='test0'
nickname='test0_textimage_svm'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



In [ ]:
eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))

# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [ ]:
img_feature_eval_list = []
for pid, name in eval_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))


In [ ]:
x_feature_list = vectorizer.transform(map(lambda i : i[1],eval_x_text_list))

In [ ]:
concat_eval_x_list = sparse.hstack((x_feature_list, img_feature_eval_list),format='csr')

In [ ]:
pred_list = clf2.predict(concat_eval_x_list)

In [ ]:

import requests
name='test0'
nickname='test0_textimage_svm'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())

